• As another example of using manifold learning for visualization, let's take a look at the MNIST handwritten digits set.

• This data is similar to the digits we saw in In-Depth: Decision Trees and Random Forests, but with many more pixels per image.

• It can be downloaded from http://mldata.org/ with the Scikit-Learn utility:

• We begin with the standard imports:

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot
import seaborn as sns; sns.set()

In [18]:
from sklearn.datasets import fetch_mldata
DATA_PATH = '/home/rajesh/git/python_excercise_programiz/python/12_scikit_learn'
mnist = fetch_mldata('MNIST original', data_home=DATA_PATH)
mnist.data.shape

(70000, 784)

• This consists of 70,000 images, each with 784 pixels (i.e. the images are 28×28).

• As before, we can take a look at the first few images: